In [ ]:
import json
import pandas as pd
from neo4j import GraphDatabase

with open('neo4j_dbinfo', 'r') as f:
    neo4j_info = json.load(f)

neo4j_uri = neo4j_info["uri"]
neo4j_user = neo4j_info["username"]
neo4j_password = neo4j_info["password"]

# === Load CSVs ===
aops_df = pd.read_csv("aops.csv")
events_df = pd.read_csv("events.csv")
ke_df = pd.read_csv("ke-relationships.csv")
stressors_df = pd.read_csv("stressors.csv")

# === Connect to Neo4j ===
driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_uri, neo4j_password))

def run_query(tx, query, params=None):
    tx.run(query, params or {})

with driver.session() as session:
    ## === 1. AOPs ===
    for _, row in aops_df.iterrows():
        session.write_transaction(run_query, """
        MERGE (a:AOP {id: $id})
        SET a.title = $title,
            a.contact = $contact,
            a.license = $license,
            a.mie = $mie,
            a.ao = $ao,
            a.status = $status,
            a.project = $project
        """, {
            "id": int(row.ID),
            "title": row.Title,
            "contact": row["Point of Contact"],
            "license": row.License,
            "mie": row.MIE,
            "ao": row.AO,
            "status": row["OECD Status"],
            "project": row["OECD Project"]
        })

    ## === 2. Events ===
    for _, row in events_df.iterrows():
        session.write_transaction(run_query, """
        MERGE (e:Event {id: $id})
        SET e.title = $title,
            e.level = $level
        """, {
            "id": int(row.ID),
            "title": row.Title,
            "level": row["Biological organization"]
        })

        if pd.notna(row.AOPs):
            session.write_transaction(run_query, """
            MATCH (a:AOP), (e:Event {id: $eid})
            WHERE $aop_title CONTAINS a.title OR a.title CONTAINS $aop_title
            MERGE (a)-[:HAS_EVENT]->(e)
            """, {
                "eid": int(row.ID),
                "aop_title": row.AOPs
            })

    ## === 3. KE Relationships (KE_Relation) ===
    for _, row in ke_df.iterrows():
        rel_id = int(row.ID)
        upstream = row["Upstream Event"]
        downstream = row["Downstream Event"]

        session.write_transaction(run_query, """
        MERGE (up:Event {title: $up})
        MERGE (down:Event {title: $down})
        MERGE (rel:KE_Relation {id: $id})
        MERGE (rel)-[:CAUSES]->(down)
        MERGE (up)-[:CAUSES]->(rel)
        """, {
            "id": rel_id,
            "up": upstream,
            "down": downstream
        })

        if pd.notna(row.AOPs):
            session.write_transaction(run_query, """
            MATCH (a:AOP), (rel:KE_Relation {id: $id})
            WHERE $aop_title CONTAINS a.title OR a.title CONTAINS $aop_title
            MERGE (a)-[:HAS_RELATIONSHIP]->(rel)
            """, {
                "id": rel_id,
                "aop_title": row.AOPs
            })

    ## === 4. Stressors ===
    for _, row in stressors_df.iterrows():
        sid = int(row.ID)
        aop_id = row["Associated AOPs"]

        # Create the stressor node with aop_id as property
        session.write_transaction(run_query, """
        MERGE (s:Stressor {id: $id})
        SET s.name = $name,
            s.chemical = $chem,
            s.aop_id = $aop_id
        """, {
            "id": sid,
            "name": row["Prototypical Stressor Name"],
            "chem": row["Associated Chemical(s): Name, CAS-RN, DTXSID"],
            "aop_id": aop_id
        })

        # Link directly using aop_id = AOP.id
        if pd.notna(aop_id) and str(aop_id).isdigit():
            session.write_transaction(run_query, """
            MATCH (a:AOP {id: toInteger($aop_id)}), (s:Stressor {id: $sid})
            MERGE (s)-[:ASSOCIATED_WITH]->(a)
            """, {
                "aop_id": int(aop_id),
                "sid": sid
            })



In [ ]:
import pandas as pd
from neo4j import GraphDatabase

stressors_df = pd.read_csv("stressors.csv")

driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_uri, neo4j_password))

def run_query(tx, query, params=None):
    tx.run(query, params or {})

with driver.session() as session:
    ## === 4. Stressors ===
    for _, row in stressors_df.iterrows():
        sid = int(row.ID)
        aop_id = int(float(row["Associated AOPs"])) if pd.notna(row["Associated AOPs"]) else None

        # Create the stressor node with aop_id as property
        session.write_transaction(run_query, """
        MERGE (s:Stressor {id: $id})
        SET s.name = $name,
            s.chemical = $chem,
            s.aop_id = $aop_id
        """, {
            "id": sid,
            "name": row["Prototypical Stressor Name"],
            "chem": row["Associated Chemical(s): Name, CAS-RN, DTXSID"],
            "aop_id": aop_id
        })

        # Link directly using aop_id = AOP.id
        if pd.notna(aop_id) and str(aop_id).isdigit():
            session.write_transaction(run_query, """
            MATCH (s:Stressor), (a:AOP)
            WHERE toInteger(s.aop_id) = a.id
            MERGE (s)-[:ASSOCIATED_WITH]->(a)
            """, {
                "aop_id": int(aop_id),
                "sid": sid
            })

In [ ]:
import pandas as pd
from neo4j import GraphDatabase

aops_df = pd.read_csv("aops.csv")
events_df = pd.read_csv("events.csv")
ke_df = pd.read_csv("ke-relationships.csv")
stressors_df = pd.read_csv("stressors.csv")

# === Connect to Neo4j ===
driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_uri, neo4j_password))

def run_query(tx, query, params=None):
    tx.run(query, params or {})

with driver.session() as session:
    ## === 1. AOPs ===
    for _, row in aops_df.iterrows():
        session.write_transaction(run_query, """
        MERGE (a:AOP {id: $id})
        SET a.title = $title,
            a.contact = $contact,
            a.license = $license,
            a.mie = $mie,
            a.ao = $ao,
            a.status = $status,
            a.project = $project
        """, {
            "id": int(row.ID),
            "title": row.Title,
            "contact": row["Point of Contact"],
            "license": row.License,
            "mie": row.MIE,
            "ao": row.AO,
            "status": row["OECD Status"],
            "project": row["OECD Project"]
        })

In [ ]:
import pandas as pd
from neo4j import GraphDatabase

aops_df = pd.read_csv("aops.csv")
events_df = pd.read_csv("events.csv")
ke_df = pd.read_csv("ke-relationships.csv")
stressors_df = pd.read_csv("stressors.csv")

driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_uri, neo4j_password))

def run_query(tx, query, params=None):
    tx.run(query, params or {})

with driver.session() as session:    
    ## === 3. KE Relationships (KE_Relation) ===
    for _, row in ke_df.iterrows():
        rel_id = int(row.ID)
        upstream = row["Upstream Event"]
        downstream = row["Downstream Event"]
        creation_date = row["Creation Date"]
        last_updated = row["Last Updated"]
        aops = row["AOPs"]

        print(upstream, downstream)

        session.write_transaction(run_query, """
        MATCH (n:KE_Relation {id: $id})
        SET n.upstream_event = $up,
            n.downstream_event = $down,
            n.last_updated = $last_updated,
            n.creation_date = $creation_date,
            n.AOPs = $aops
        RETURN n
        """, {
            "id": rel_id,
            "up": upstream,
            "down": downstream,
            "creation_date": creation_date,
            "last_updated": last_updated,
            "aops" : aops
        })

        if pd.notna(row.AOPs):
            session.write_transaction(run_query, """
            MATCH (a:AOP), (rel:KE_Relation {id: $id})
            WHERE $aop_title CONTAINS a.title OR a.title CONTAINS $aop_title
            MERGE (a)-[:HAS_RELATIONSHIP]->(rel)
            """, {
                "id": rel_id,
                "aop_title": row.AOPs
            })


In [ ]:
import pandas as pd
from neo4j import GraphDatabase

aops_df = pd.read_csv("aops.csv")
events_df = pd.read_csv("events.csv")
ke_df = pd.read_csv("ke-relationships.csv")
stressors_df = pd.read_csv("stressors.csv")

# === Connect to Neo4j ===
driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_uri, neo4j_password))

def run_query(tx, query, params=None):
    tx.run(query, params or {})

with driver.session() as session:    

    ## === 2. Events ===
    for _, row in events_df.iterrows():
        session.write_transaction(run_query, """
        MERGE (e:Event {id: $id})
        SET e.title = $title,
            e.biological_organization = $biological_organization,
            e.creation_date = $creation_date,
            e.last_updated = $last_updated,
            e.AOPs = $aops                                         
        """, {
            "id": int(row.ID),
            "title": row.Title,
            "biological_organization": row["Biological organization"],
            "creation_date": row["Creation Date"],
            "last_updated": row["Last Updated"],
            "aops": row["AOPs"],
        })

        if pd.notna(row.AOPs):
            session.write_transaction(run_query, """
            MATCH (a:AOP), (e:Event {id: $eid})
            WHERE $aop_title CONTAINS a.title OR a.title CONTAINS $aop_title
            MERGE (a)-[:HAS_EVENT]->(e)
            """, {
                "eid": int(row.ID),
                "aop_title": row.AOPs
            })